---
title: "Is Pydantic making your LLM dumber?"
date: "09/15/2024"
date-modified: last-modified
description-meta: "Pydantic is a powerful tool for data validation, but can it make your LLM dumber?"
toc: true
toc-depth: 3
lightbox: true
fig-cap-location: margin
categories:
  - llm
  - openai
  - pydantic
  - python
author:
  - name: Dylan Castillo
    url: https://dylancastillo.co
    affiliation: Iwana Labs
    affiliation-url: https://iwanalabs.com
citation: true
comments:
  utterances:
    repo: dylanjcastillo/blog_comments
    theme: dark-blue
    issue-term: pathname
---

In [101]:
import pandas as pd

df = pd.read_json("../data/live_bench/coding/question.jsonl", lines=True, orient="records")
df = df.query("livebench_release_date == '2024-06-24'")
df.shape

(88, 14)

In [103]:
df.task.unique()

array(['coding_completion', 'LCB_generation'], dtype=object)

In [97]:
# Verify there's a single turn in all rows
assert all(len(turns) == 1 for turns in df.turns), "There should be a single turn in all rows"

In [133]:
#| output: false
#| echo: false

# Extract the full prompt from the turns
df["prompt"] = df.turns.apply(lambda x: x[0]).str.strip()

# Split prompt by ### and extract the first part
df["first_message"] = df.prompt.str.split("###").str[1].str.strip()


In [132]:
#| output: false
#| echo: false

with pd.option_context('display.max_colwidth', None):
    display(df[["task", "first_message"]].drop_duplicates().T)

,0,38
task,coding_completion,LCB_generation
first_message,"Instructions: You are an expert Python programmer. You will be given a question (problem specification) and the first lines of Python solution to this problem, and will write in Python the remaining lines of the program to produce a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the second part of the program that you wrote.",Instructions: You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the program.
